# MIMIC Pre-Processing Example
An example of how to prepare MIMIC data for use in a machine learning algorithm.  Takes the raw table data and creates a single table of static (don't change with time) and dynamic (do change with time) data features for the MIMIC patient population.  The outcome label used is mortality.

## Notebook Structure Overview
This notebook is organized into several main sections, each dedicated to a specific part of the data pre-processing workflow for the MIMIC dataset. The sections are as follows:
1. **Data Loading**: Importing the necessary datasets from MIMIC.
2. **Data Cleaning and Transformation**: Preparing the data for analysis, including cleaning and transforming data.
3. **Feature Engineering**: Creating new features from the existing data to better capture the insights needed for the analysis.
4. **Data Integration**: Combining various datasets into a unified format suitable for machine learning models.
5. **Final Preparations**: Finalizing the dataset for use in machine learning, including splitting into training and test sets.
Each section is marked with a heading and introductory text to explain its purpose and contents.

## Introduction
This notebook demonstrates the pre-processing steps required to prepare MIMIC data for machine learning analysis. It focuses on assembling a cohesive dataset from the raw MIMIC tables, categorizing features into static and dynamic ones, and defining mortality as the outcome of interest.

In [ ]:
import numpy as np
import pandas as pd
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
import os
import pickle 
from tqdm.auto import tqdm, trange
from tqdm.notebook import tqdm
tqdm.pandas()

In [ ]:
dataDirStr = 'MIMIC_Codes/MIMIC_3/Csv/'   # MIMIC CSV file location.
cacheDirStr = 'MIMIC_Codes/MIMIC_3/cache/'  # Cache directory for intermediate files.

## Timestamped Events
---
Start by gathering all of our events of interest that have a time stamp.

### Service Type

In [ ]:
srvTbl = pd.read_csv(dataDirStr + 'SERVICES.csv')
srvTbl.columns = srvTbl.columns.str.lower()

In [ ]:
dateCols = [ 'transfertime' ]
convert_date_type(srvTbl,dateCols)

In [ ]:
columnMap = {
    'subject_id': 'SubjectId', 
    'transfertime': 'Date',
    'curr_service': 'ServiceType'
}

In [ ]:
def select_mimic_columns(tbl,mapper):
    d = {}
    for k in mapper.keys():
        d[mapper[k]] = tbl[k]
    return pd.DataFrame(d)      

In [ ]:
events = select_mimic_columns(srvTbl,columnMap)

### Admission Type

In [ ]:
admitTbl = pd.read_csv(dataDirStr + 'ADMISSIONS.csv')
admitTbl.columns = admitTbl.columns.str.lower()

In [ ]:
admitTbl = admitTbl.sort_values(by=['subject_id', 'admittime'])  #Keeping only first admissions
admitTbl = admitTbl.drop_duplicates(subset='subject_id', keep='first')

In [ ]:
unique_subject_ids = admitTbl['subject_id'].nunique()

In [ ]:
columnMap = {
    'subject_id': 'SubjectId', 
    'admittime': 'Date',
    'admission_type': 'AdmitType'
}

In [ ]:
eventsNew = select_mimic_columns(admitTbl,columnMap)

In [ ]:
events = pd.concat([ events, eventsNew ], ignore_index=True, sort=False)

In [ ]:
events

### Chart Events

`CHARTEVENTS` is a massive table so only extract the events we're interested in.  Use the dask library to implement parallelized filtering of the table as it's read in from its csv file.

In [ ]:
chartEvents = {
    'GscVerbal': [ 723, 223900 ],
    'GscMotor': [ 454, 223901 ],
    'GscEyes': [ 184, 220739 ],
    'SystolicBloodPressure': [ 51, 442, 455, 6701, 220050, 220179 ],
    'HeartRate': [ 211, 220045 ],
    'Temperature': [ 676, 678, 223762, 223761 ],
    'BloodO2': [ 190, 3420, 3422, 223835 ]
}
    

In [ ]:
allEvents = [ ]
for k in chartEvents.keys():
    allEvents += chartEvents[k]

In [ ]:
renameMap = {
    'subject_id': 'SubjectId', 
    'charttime': 'Date'
}

In [ ]:
def extract_multiple_events(tbl,renameMap,eventMap,eventIds,value,):
    n = len(tbl.index)
    d = {}
    
    for k in renameMap.keys():
        d[ renameMap[k] ] = tbl[k]
    
    for k in eventMap.keys():
        d[k] = pd.Series(np.nan,index=tbl.index)
        eventIdx = tbl[eventIds].isin(eventMap[k])
        d[k].loc[eventIdx] = tbl[value].loc[eventIdx]
     
    return pd.DataFrame(d)      

In [ ]:
eventsNew = extract_multiple_events(chartTbl,renameMap=renameMap,eventMap=chartEvents,eventIds='itemid',value='value')

In [ ]:
unique_valuesV = eventsNew['GscVerbal'].unique()

In [ ]:
replace_dict = {
    '5 Oriented': 5,
    '1.0 ET/Trach': 1,
    '4 Confused': 4,
    '2 Incomp sounds': 2,
    '1 No Response': 1,
    '3 Inapprop words': 3,
    'No Response-ETT': 1,
    'Oriented': 5,
    'No Response': 0,
    'Confused': 4,
    'Incomprehensible sounds': 2,
    'Inappropriate Words': 3
}

eventsNew['GscVerbal'] = eventsNew['GscVerbal'].replace(replace_dict)

eventsNew['GscVerbal'] = eventsNew['GscVerbal'].fillna(0)

In [ ]:
unique_valuesV = eventsNew['GscVerbal'].unique()

In [ ]:
replace_dict_motor = {
    '6 Obeys Commands': 6,
    '5 Localizes Pain': 5,
    '1 No Response': 1,
    '4 Flex-withdraws': 4,
    '2 Abnorm extensn': 2,
    '3 Abnorm flexion': 3,
    'Localizes Pain': 5,
    'Obeys Commands': 6,
    'Flex-withdraws': 4,
    'No response': 1,
    'Abnormal Flexion': 3,
    'Abnormal extension': 2,
    'nan':0
}

eventsNew['GscMotor'] = eventsNew['GscMotor'].replace(replace_dict_motor)

eventsNew['GscMotor'] = eventsNew['GscMotor'].fillna(0)

In [ ]:
eventsNew['GscMotor'] = eventsNew['GscMotor'].replace("NaN", 0)

In [ ]:
unique_valuesM = eventsNew['GscMotor'].unique()

In [ ]:
replace_dict_eyes = {
    '4 Spontaneously': 4,
    '1 No Response': 1,
    '2 To pain': 2,
    '3 To speech': 3,
    'To Speech': 3,
    'Spontaneously': 4,
    'To Pain': 2,
    'nan':0
}

eventsNew['GscEyes'] = eventsNew['GscEyes'].replace(replace_dict_eyes)

eventsNew['GscEyes'] = eventsNew['GscEyes'].replace("NaN", 0)

eventsNew['GscEyes'] = eventsNew['GscEyes'].fillna(0)

In [ ]:
unique_valuesE = eventsNew['GscEyes'].unique()

In [ ]:
events = pd.concat([ events, eventsNew ], ignore_index=True, sort=False)

### Output Events

In [ ]:
output = pd.read_csv(dataDirStr + 'OUTPUTEVENTS.csv')
output.columns = output.columns.str.lower()

In [ ]:
dateCols = [ 'charttime' ]
convert_date_type(output,dateCols)

In [ ]:
outputEvents = {
    'Urine': [ 40055, 43175, 40069, 40715, 40473, 40085, 40057, 40056, 40405, 40428, 40086, 40096, 
             40651, 226559, 226560, 226561, 226584, 226563, 226564, 226565, 226567, 226557, 226558, 
             227488]
}

In [ ]:
renameMap = {
    'subject_id': 'SubjectId', 
    'charttime': 'Date'
}

In [ ]:
output = output.loc[output.itemid.isin(outputEvents['Urine'])]

In [ ]:
eventsNew = extract_multiple_events(output,renameMap=renameMap,eventIds='itemid',value='value',eventMap=outputEvents)

In [ ]:
events = pd.concat([ events, eventsNew ], ignore_index=True, sort=False)

### Adjust Data Column Types
Cast numerical columns to float and convert categorical string columns to categorical floating point values.

In [ ]:
catColumns = [ 'ServiceType', 'AdmitType', 'GscVerbal', 'GscMotor', 'GscEyes' ]
numColumns = [ 'SystolicBloodPressure', 'HeartRate', 'Temperature', 'BloodO2', 'Urine' ]

In [ ]:
catMap = {}
for c in tqdm(catColumns):
    vals = events[c].loc[~events[c].isna()].unique()
    nums = list(range(len(vals)))
    catNums = dict(zip(vals,nums))
    
    catMap[c] = catNums
    
    notNa = ~events[c].isna()
    events.loc[notNa,c] = events[c].loc[notNa].map(catNums)
    events[c] = events[c].astype(float)

In [ ]:
for c in tqdm(numColumns):
    events.loc[events[c] == '<NA>',c] = np.nan
    events[c] = events[c].astype(float)

In [ ]:
fp = open(cacheDirStr + 'CategoricalVariableMapping.pkl', 'wb')
pickle.dump(catMap, fp)
fp.close()

### Normalize Time
Express time as the number of seconds relative to the first event in the data (usually an admission event).

In [ ]:
def calc_elapsed_time(tbl):
    tbl = tbl.sort_values(by='Date')
    dataCols = tbl.columns[2:]
    
    timeSec = tbl.Date - tbl.Date.min()
    timeSec.rename('Time',inplace=True)
    
    return pd.concat([ timeSec, tbl[dataCols] ],axis=1)

In [ ]:
events = events.groupby('SubjectId').progress_apply(calc_elapsed_time)

## Static Features
---
The following features are static for the duration of the data time span.

### Age
Due to the random offsets applied to each patient's timeline, age is calculated relative to date of first admission.

In [ ]:
patientTbl = pd.read_csv(dataDirStr + 'PATIENTS.csv')
patientTbl.columns = patientTbl.columns.str.lower()

In [ ]:
dateCols = [ 'dob', 'dod', 'dod_hosp', 'dod_ssn' ]
convert_date_type(patientTbl,dateCols)

In [ ]:
def find_first_admission(tbl):
    return tbl.sort_values(by='admittime').iloc[0][[ 'hadm_id', 'admittime' ]]

In [ ]:
firstAdmit = admitTbl.groupby('subject_id').progress_apply(find_first_admission)

In [ ]:
patientIds = patientTbl.subject_id
patientAdmitTime = firstAdmit.loc[patientTbl.subject_id].admittime
patientDob = pd.Series(patientTbl.dob.values,index=patientTbl.subject_id)
age = pd.DataFrame({ 'Age': patientAdmitTime.dt.year - patientDob.dt.year})

In [ ]:
staticFeatures = age

### Disease Diagnoses

In [ ]:
diagnoses = pd.read_csv(dataDirStr + 'DIAGNOSES_ICD.csv')
diagnoses.columns = diagnoses.columns.str.lower()

In [ ]:
firstAdmitDiag = diagnoses.hadm_id.isin(firstAdmit.hadm_id)
diagnoses = diagnoses.loc[firstAdmitDiag]

In [ ]:
icdDict = pd.read_csv(dataDirStr + 'D_ICD_DIAGNOSES.csv')
icdDict.columns = icdDict.columns.str.lower()
icdDict = icdDict.sort_values(by = 'icd9_code', ascending=True)

In [ ]:
diseaseCodes = {
    'Aids': [ '042' ]
}  

**Musa:** I use a different approach to select the ICD codes corresponding to malignancy.  These are based on the [ICD9 wikipedia page](https://en.wikipedia.org/wiki/List_of_ICD-9_codes) and should encompass both malignant cancers and malignant blood disorders.

In [ ]:
malignantPrefix = [ '14', '15', '16', '17', '18', '19', '20' ]

In [ ]:
cancerIcd = []
for pfx in malignantPrefix:
    idx = icdDict.icd9_code.str.contains(f'^{pfx}')
    cancerIcd += list(icdDict.loc[idx].icd9_code)
diseaseCodes['Cancer'] = cancerIcd    

In [ ]:
def check_diagnoses(tbl,diagCode,diseaseCodes):
    diag = {}
    for d in diseaseCodes.keys():
        diag[d] = tbl[diagCode].isin(diseaseCodes[d]).sum() > 0
    return pd.Series(diag)

In [ ]:
diagScan = diagnoses.groupby('subject_id').progress_apply(
    check_diagnoses,diagCode='icd9_code',diseaseCodes=diseaseCodes
)

In [ ]:
staticFeatures = pd.concat([ staticFeatures, diagScan ], axis=1)

## Merge Static and Timestamped Features

In [ ]:
idx = events.index.get_level_values(0)

In [ ]:
staticRepeatedTbl = staticFeatures.loc[idx]
staticRepeatedTbl.index = events.index

In [ ]:
events = pd.concat([ events, staticRepeatedTbl ],axis=1)

In [ ]:
events

## Label Data

In [ ]:
deathTbl = pd.DataFrame({ 'Death': patientTbl.expire_flag.values != 0 },index=patientTbl.subject_id)

In [ ]:
idx = events.index.get_level_values(0)
events['Death'] = deathTbl.loc[idx].Death.values

In [ ]:
events

In [ ]:
events

In [ ]:
events.to_parquet(cacheDirStr + 'ElapsedTimeEvents.parquet', engine='pyarrow')